In [21]:
import os
DATA_DIR = "data"
ANNOTATION_TRAIN_DIR = os.path.join(DATA_DIR, "annotations/train")
AROUSAL_TRAIN_DIR = os.path.join(DATA_DIR, "annotations/train/arousal")
VALENCE_TRAIN_DIR = os.path.join(DATA_DIR, "annotations/train/valence")
AROUSAL_TRAIN_FILES = os.path.join(AROUSAL_TRAIN_DIR, "*.txt")
VALENCE_TRAIN_FILES = os.path.join(VALENCE_TRAIN_DIR, "*.txt")

BBOXES_TRAIN_DIR = os.path.join(DATA_DIR, "bboxs/train")
LANDMARKS_TRAIN_DIR = os.path.join(DATA_DIR, "landmarks/train")
VIDEOS_TRAIN_DIR = os.path.join(DATA_DIR, "videos/train")
VIDEOS_TEST_DIR = os.path.join(DATA_DIR, "videos/test")

In [17]:
"""
Sanity: if the number of annotations same as the number of frames.
"""
import os

import numpy as np
import cv2

def count_frame(path: str) -> int:
    video_obj = cv2.VideoCapture(path)
    frame_counter = 0
    success = 1
    while success:
        success, _ = video_obj.read()
        frame_counter += 1
    return frame_counter - 1

def count_lines(path: str) -> int:
    line_counter = 0
    with open(path) as f:
        for _ in f:
            line_counter += 1
    return line_counter
num_frames = count_frame(os.path.join(VIDEOS_TRAIN_DIR, "105.avi"))
num_annotations = count_lines(os.path.join(AROUSAL_TRAIN_DIR, "105.txt"))
print(num_frames)
print(num_annotations)
assert num_frames == num_annotations

10076
10076


In [76]:
"""
Filter frames by arousal and valence and save to a folder.
"""
import glob
from collections import defaultdict

import cv2

def is_bored(valence: float, 
             arousal: float, 
             radius: float=0.1) -> bool:
    """
    Defined bored in valence, arousal. See Fig. 1 in https://arxiv.org/pdf/1804.10938.pdf.
    """
    bored_valence = -0.34 
    bored_arousal = -0.78
    return (bored_valence - radius <= valence <= bored_valence + radius and 
            bored_arousal - radius <= arousal <= bored_arousal + radius)

def get_video_path(video_id: str):
    mp4_path = os.path.join(VIDEOS_TRAIN_DIR, video_id + ".mp4")
    avi_path = os.path.join(VIDEOS_TRAIN_DIR, video_id + ".avi")
    if os.path.isfile(mp4_path):
        return mp4_path
    if os.path.isfile(avi_path):
        return avi_path
    raise ValueError("Video {} does not exist".format(video_id))

def create_candidates(output_dir="candidates", radius: float=0.2) -> str:
    candidate_frames = defaultdict(set)
    total_count = 0
    bored_count = 0
    for arousal_file, valence_file in zip(sorted(glob.glob(AROUSAL_TRAIN_FILES)), sorted(glob.glob(VALENCE_TRAIN_FILES))):
        video_id = os.path.splitext(os.path.basename(arousal_file))[0] # eg. file_id = 105
        arousal_file = open(arousal_file, 'r')
        valence_file = open(valence_file, 'r')
        arousal_values = [float(line) for line in arousal_file.readlines()]
        valence_values = [float(line) for line in valence_file.readlines()]
        for frame_id, (valence_value, arousal_value) in enumerate(zip(valence_values, arousal_values)):
            total_count += 1
            if is_bored(valence_value, arousal_value, radius=radius):
                bored_count += 1
                candidate_frames[video_id].add(frame_id)

    # Create frame file for each candidate frames.
    output_dir = os.path.join(output_dir, str(radius))
    os.makedirs(output_dir, exist_ok=True)
    for video_id, candidate_frames in candidate_frames.items():
        video_file = get_video_path(video_id)
        video_obj = cv2.VideoCapture(video_file)
        frame_id = 0
        success = 1
        while success:
            success, frame = video_obj.read()
            if frame_id in candidate_frames:
                cv2.imwrite(os.path.join(output_dir, "{}_{}.jpg".format(video_id, frame_id)),
                            frame)
            frame_id += 1

    return candidate_frames, bored_count, total_count
    
    
candidate_frames, bored_count, total_count = create_candidates(radius=0.3)

4872
1008653
{2560, 2561, 2562, 2563, 2564, 2565, 2566, 2567, 2568, 2569, 2570, 2571, 1295, 1296, 1297, 1298, 1299, 1300, 1301, 1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309, 1310, 1311, 1312, 1313, 1314, 1315, 1316, 1317, 1318, 1319, 1320, 1321, 1322, 1323, 1324, 1325, 1326, 1327, 1328, 1329, 1330, 1331, 1332, 1359, 1360, 1361, 1362, 1363, 1364, 1365, 1366, 1367, 1368, 1369, 1370, 1371, 1372, 1373, 1374, 1375, 1376, 1377, 2451, 2452, 2453, 2454, 2455, 2456, 2457, 2458, 2459, 2460, 2461, 2462, 2463, 2464, 2465, 2466, 2467, 2468, 2469, 2470, 2471, 2472, 2473, 2474, 2475, 2476, 2477, 2478, 2479, 2480, 2481, 2482, 2483, 2484, 2485, 2486, 2487, 2488, 2489, 2490, 2491, 2492, 2493, 2494, 2495, 2496, 2497, 2542, 2543, 2544, 2545, 2546, 2547, 2548, 2549, 2550, 2551, 2552, 2553, 2554, 2555, 2556, 2557, 2558, 2559}


In [ ]:
"""
Radius = 0.3, annotation. Inclusive range.
"""
annotations = [
    ["299_448", "299_454"],
    ["305_1712", "305_1760"],
    ["308_247", "308_286"],
    ["312_1289", "312_1329"],
    ["313_324", "313_345"],
    ["313_1276", "313_1625"],
    ["369_680", "369_896"],
    ["377_2034", "377_2234"], 
    ["385_244", "385_521"],
    ["386_84", "386_128"],
    ["389_418", "389_3404"],
    ["400_42", "400_72"],
    ["447_929", "447_1118"],
    ["448_1295", "448_2571"]
]